In [1]:
import os
import gzip
import itertools
import sys
import json
import glob
from tqdm.auto import tqdm
import numpy as np
import pycocotools.mask as mask_util
import io
import contextlib
from multiprocessing import Pool
import pickle
from scipy.stats import gmean
from scipy.special import softmax
from collections import defaultdict

In [2]:
PATH_TO_ANNS = "../data/val_annot.json"
PATH_TO_PREDS = "/scratch/shared/beegfs/prannay/ego4d_data/ckpt/results/traces_v2_all/vq_stats_val_{}.json.gz"
PATH_TO_DINO = "/scratch/shared/beegfs/prannay/ego4d_data/ckpt/results/traces_v2_all/vq_stats_val_{}_dino_scores_all.pkl"
assert os.path.exists(PATH_TO_ANNS)

In [3]:
with open(PATH_TO_ANNS, "r") as f:
    anns = json.load(f)

In [4]:
PRED_IDXS = range(0, 200)
path_to_preds = [PATH_TO_PREDS.format(i) for i in PRED_IDXS]
assert all([os.path.exists(p) for p in path_to_preds])
path_to_dinos = [PATH_TO_DINO.format(i) for i in PRED_IDXS]
assert all([os.path.exists(p) for p in path_to_dinos])

In [5]:
def preprocess_data(path):
    with gzip.open(path, "r") as f:
        data = json.load(f)
    data = data['predictions']
    # convert generic Dict[List] to List[Dict]
    out_list = []
    keys = sorted(data.keys())
    for idx in range(len(data[keys[0]])):
        save_dict = {k: data[k][idx] for k in keys}
        save_dict['dataset_uid'] = save_dict['dataset_uids']
        del save_dict['dataset_uids'] 
        out_list.append(save_dict)
    return out_list

In [6]:
def preprocess_dino(path):
    with open(path, "rb") as f:
        dino_data = pickle.load(f)
    return dict(dino_data)

In [11]:
def run_inner_episode(inputs):
    pred, gt, dino = inputs
    gt_fnos = [a['frame_number'] for a in gt['response_track']]
    pred_fnos = [a['frame_number'] for a in pred['groundtruth_response_tracks']]
    # print(len(pred['groundtruth_response_tracks']))
    assert gt_fnos == pred_fnos
    assert gt_fnos == list(range(min(gt_fnos), max(gt_fnos) + 1))
    assert (gt['dataset_uid'] == pred['dataset_uid'])
    # dino is a list
    import pdb; pdb.set_trace()
    gt_fnos = set(gt_fnos)
    gt_list = defaultdict(dict)
    assert len(dino) == len(pred['predicted_bboxes']) == len(pred['predicted_scores'])
    for d, bb, sc in zip(dino, pred['predicted_bboxes'], pred['predicted_scores']):
        if d['frame_number'] in gt_fnos:
            gt_list.append((d['frame_number'], bb, sc, d['dino_scores']))

In [12]:
def run_outer_episode(i, duid2gt=None):
    pred_path = PATH_TO_PREDS.format(i)
    dino_path = PATH_TO_DINO.format(i)
    save_path = pred_path.replace(".json.gz", "_sc_dino_gmean_gt_peak_tracker_init.json")
    if os.path.exists(save_path):
        return
    pred = preprocess_data(pred_path)
    dino = preprocess_dino(dino_path)
    duid2pred = {p['dataset_uid']: p for p in pred}
    valid_duids = set([ann['dataset_uid'] for ann in duid2gt.values() if ann['clip_uid'] is not None])
    duid2pred = {k: v for k, v in duid2pred.items() if k in valid_duids}
    duid2gt = {k: v for k, v in duid2gt.items() if k in duid2pred.keys()}
    duid2dino = {duid: dino[duid] for duid in duid2pred}
    assert len(duid2pred) == len(duid2gt) == len(duid2dino)
    inputs = [(duid2pred[k], duid2gt[k], duid2dino[k]) for k in sorted(duid2pred.keys())]
    outputs = [run_inner_episode(inp) for inp in inputs]
    with open(save_path, "w") as f:
        json.dump(outputs, f)

In [13]:
duid2gt = {g['dataset_uid']: g for g in anns if g['clip_uid'] is not None}
run_outer_episode(0, duid2gt=duid2gt)

--Return--
None
> /tmp/ipykernel_66517/704524619.py(10)run_inner_episode()
      6     assert gt_fnos == pred_fnos
      7     assert gt_fnos == list(range(min(gt_fnos), max(gt_fnos) + 1))
      8     assert (gt['dataset_uid'] == pred['dataset_uid'])
      9     # dino is a list
---> 10     import pdb; pdb.set_trace()

[1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350]
8
1433
1433
1433
1433
1433

1433
1433
1433
